<a href="https://colab.research.google.com/github/eipshita/Cyberbullying-Detection-for-Chakma-Language-on-Social-Media/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!pip install scikit-multilearn

# to convert emojis to text
!pip install emot
!pip install xgboost
!pip install Textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import numpy as np 
import pandas as pd
import re
import gc
import os
import fileinput
import string
import tensorflow as tf
import zipfile
import datetime
import sys
from tqdm  import tqdm
tqdm.pandas()
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers import Embedding
from sklearn.metrics import classification_report
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


from skmultilearn.ensemble import MajorityVotingClassifier
from skmultilearn.cluster import FixedLabelSpaceClusterer
from skmultilearn.problem_transform import ClassifierChain
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [4]:
!pip install emoji

import emoji

def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 240 kB 5.0 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=fc4cb4dd21ffc908e547fd4f446f4d170fff8d1234e9c94e5f265f40ee196567
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [8]:
# import libraries and packages

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.feature_extraction.text import  CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import VotingClassifier 
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV 

from textblob import TextBlob

import nltk.data
import numpy as np 
import pandas as pd
import re
import time
import string

# pd.set_option("max_colwidth" ,220)
# wn = nltk.WordNetLemmatizer()
# nltk.download('wordnet')
# nltk.download('stopwords')
# stopwords = nltk.corpus.stopwords.words('english')

In [9]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/English Data.csv',encoding='UTF-8')

data

,Comments,Posts,Source,English Translation,Category,100%,Annotation
0,nomonsuk,NaN,Facebook,third gender,men/boy,100.0,Bully
1,morahaji sibi re chok padai di i,NaN,Facebook,someone wishing other person to die who likes ...,men/boy,50.0,Neutral
2,Ettun para doll mili sok nw pelede i,NaN,Facebook,don't you find a beautiful girl rather than th...,woman/girl,30.0,Not Bully
3,huccor huttun ta chul an digile o mile mudi nw...,NaN,Facebook,his hair is mixed product/hybrid that no one w...,men/boy,80.0,Bully
4,tobbabe,NaN,Facebook,your father,men/boy,100.0,Bully
...,...,...,...,...,...,...,...
1012,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1013,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1014,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1015,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Bengali Data.csv',encoding='UTF-8')

data

,Comments,Posts,Source,English Translation,Category,100%,Annotation
0,আধিবাসী না দাদা উপজাতি,NaN,Facebook,"brother, it's tribal not indegeneous",community,100,Bully
1,প্রথমে মনে গোজ্জোং লাগেজ বুক্কে,NaN,Tiktok,first I thought she was carrying lugguage(mock...,woman/girl,60,Bully
2,জুম্ম হাপ্পো,NaN,Facebook,shifting cultivator who eats snake,community + men/boy,80,Bully
3,চিদোলি,NaN,Facebook,woman who eats ferminated fishpaste(fishpaste ...,women/girls,100,Bully
4,মেদে রা হও হি দেগিনে লাং তগহ,NaN,Facebook,"eat stool,what you find connection between them",men/boys,100,Bully
...,...,...,...,...,...,...,...
100,"পাগারালোই পিঁয়াজ হুদোস তরে হন্না লব, গিন গরে পারা",NaN,Facebook,"you cut onions with saw who will marry you, yo...",men/boys,50,Neutral
101,"তুই এ পাপী নয়,বানা প্রাণী হত্যা গরচ্ছি",NaN,Facebook,"you are very sinful,you only kill animals",men/boys,50,Neutral
102,NaN,বেজ গরের,Facebook,She is doing excessive,women/girls,50,Neutral
103,ইয়ুন দ বেক্কুন জগতগুরু বুদ্ধোবুর শিষ‍্য,NaN,Facebook,these are the disciple of the world guru Buddha,religion,100,Bully


In [11]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/total dataset.csv',encoding='UTF-8')

data

,Comments,Posts,Source,English Translation,Category,100%,Annotation
0,nomonsuk,NaN,Facebook,third gender,men/boy,100,Bully
1,morahaji sibi re chok padai di i,NaN,Facebook,someone wishing other person to die who likes ...,men/boy,50,Neutral
2,Ettun para doll mili sok nw pelede i,NaN,Facebook,don't you find a beautiful girl rather than th...,woman/girl,30,Not Bully
3,huccor huttun ta chul an digile o mile mudi nw...,NaN,Facebook,his hair is mixed product/hybrid that no one w...,men/boy,80,Bully
4,আধিবাসী না দাদা উপজাতি,NaN,Facebook,"brother, it's tribal not indegeneous",community,100,Bully
...,...,...,...,...,...,...,...
1096,mogodar luccho,NaN,Facebook,you dead pervert,men/ boys,80,Bully
1097,dol uiye sudirbei,NaN,Facebook,good job motherfucker,men/boys,100,Bully
1098,manji dormo gorodon tui bana pap ani gorotte,NaN,Facebook,all of them are doing good deeds and you are d...,men/women,50,Neutral
1099,puro bandor o dok gasottun jam dede,NaN,Facebook,you jumped like a monkey from the tree,men/boys,70,Bully


In [12]:
data.columns

Index(['Comments', 'Posts', 'Source', 'English Translation', 'Category',
       '100%', 'Annotation'],
      dtype='object')

In [13]:
import string
import re


def clean_text(text):
    #text = ' '.join([word for word in text.split() if not word.startswith('@')])           #user name with @ is removed
    text = re.sub('@[a-zA-Z0-9]*', '', str(text))
    stop_words = open("/content/drive/MyDrive/Colab Notebooks/stopwords.txt" ,encoding='utf=8').read().split('\n')
    text = "".join([word.lower() for word in text if word not in string.punctuation]) 
    text = "".join([word for word in text if not word.isdigit()])
    tokens = re.split('\s+',text)
    text = " ".join(word for word in tokens if word not in stop_words)
    return text



data_train= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/total dataset.csv",header=None,sep='\t')
#data_dev= pd.read_csv("/content/drive/MyDrive/new abusive-20220218T023446Z-001/new abusive/ta-misogyny-test.csv", header=None,sep='\t')
#data_test= pd.read_csv("/content/drive/MyDrive/commaaticon_test_data (1)/test_hin.tsv", sep='\t')
data_train.columns=['Comments','Annotation']
#data_dev.columns=['Text']
data_train.head()

ValueError: ignored

In [ ]:
data_train['Annotation'].value_counts()